<a href="https://colab.research.google.com/github/ToumaTanaka/Data_Science/blob/main/Mathematical_Optimization/Allocation_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 割当問題
* 0-1整数計画問題として解く(解ベクトルxの各要素を0または1のみに限定したもの)
* PythonのソルバーであるPuLPを利用する
* 割り当てた時の目的関数が最小になる組み合わせを探す

In [2]:
!pip install pulp

     |████████████████████████████████| 41.2 MB 75 kB/s 


In [38]:
import pulp
import pandas as pd

In [39]:
#トラックを20台と出勤者を20人とした時のデータを用意
#scoreは資料の(出勤者の累積仕事量)×(トラックの仕事量)＋(出勤者の累積取上)×(トラックの取上)の仮想的な数値だとする

df = pd.read_csv('random_score.csv')
df.head()

,Track,driver,score
0,1,1,73
1,1,2,34
2,1,3,26
3,1,4,42
4,1,5,37


In [40]:
df['Var'] = [pulp.LpVariable(f'x{df.Track[L]}_{df.driver[L]}',cat="Binary") for L in df.index]
df.head()

,Track,driver,score,Var
0,1,1,73,x1_1
1,1,2,34,x1_2
2,1,3,26,x1_3
3,1,4,42,x1_4
4,1,5,37,x1_5


In [41]:
#最小値探索のモデルを定義
problem = pulp.LpProblem('割り当て問題', sense=pulp.LpMinimize)

#定義したモデルに制約式を追加
#トラックとドライバーの数が違う時ダミーの数値が同じ値をおく
#ドライバー1人にトラック一台を割り当てる制約
for j, v in df.groupby('driver'):
  problem += pulp.lpSum(v.Var) == 1
#トラック一台に1人だけ割り当てる
for i, v in df.groupby('Track'):
  problem += pulp.lpSum(v.Var) == 1

#定義したモデルに目的関数を追加
problem += pulp.lpDot(df.score,df.Var)

In [42]:
#解を求める
result = problem.solve()

#最適解が求められたか表示(最適解なら'optimal'と表示される)
print(pulp.LpStatus[result])

#目的関数の値を表示
print(pulp.value(problem.objective))

#最適化結果をデータフレームに追加
df['Val'] = df.Var.apply(pulp.value)

Optimal
159.0


In [43]:
df.head()

,Track,driver,score,Var,Val
0,1,1,73,x1_1,0.0
1,1,2,34,x1_2,0.0
2,1,3,26,x1_3,0.0
3,1,4,42,x1_4,0.0
4,1,5,37,x1_5,0.0


In [44]:
print(df[df.Val > 0][['Track', 'driver', 'score']])

     Track  driver  score
9        1      10     16
35       2      16     15
56       3      17      3
65       4       6      6
88       5       9     14
103      6       4      5
122      7       3      2
150      8      11      4
167      9       8      3
184     10       5      3
211     11      12      2
220     12       1     12
258     13      19      6
274     14      15      1
281     15       2     15
306     16       7      9
332     17      13     23
359     18      20      2
377     19      18     12
393     20      14      6
